In [1]:
#!pip install pycbrf

In [88]:
import pandas as pd
import numpy as np 
from pycbrf.toolbox import ExchangeRates
import re

In [3]:
fixed_df = pd.read_csv('data/hh_fixed.csv')
fixed_df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,Не указано
1,1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,Не указано
2,2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,Не указано
3,3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,Имеется собственный автомобиль
4,4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,Не указано


In [4]:
fixed_df = fixed_df.loc[:1000]
fixed_df.shape

(1001, 13)

In [5]:
def getParameterSalary(arg, rates):
    # Сначала получаем чистое число, убираем лишние знаки
    if (type(arg) == str):
        arg_splited = arg.split(' ')
        num = float(arg_splited[0])
        curr = arg_splited[1].replace('.', '').strip()
        
#         num = re.sub(' ','',num)
#         num = re.sub('[а-яА-ЯёЁ]','',num)
#         num = re.sub('[a-zA-Z]','',num)
#         num = num.replace('.','')
#         # Получаем чисто валюту, убираем цифры
#         curr = re.sub('[0-9]','',arg)
#       curr = curr.replace('.','').strip()
        if curr == 'USD':
            num = float(num)*float(rates['USD'].rate)
        elif curr == 'KZT':
            num = float(num)*float(rates['KZT'].rate)
        elif curr=='грн':
            num = float(num)*float(rates['UAH'].rate)
        elif curr=='белруб':
            num = float(num)*float(rates['BYN'].rate)
        elif curr=='EUR':
            num = float(num)*float(rates['EUR'].rate)
        elif curr=='KGS':
            num = float(num)*float(rates['KGS'].rate)
        elif curr=='сум':
            num = float(num)*float(rates['UZS'].rate)
        elif curr=='AZN':
            num = float(num)*float(rates['AZN'].rate)
        #отредачить
        
    salary= int(num)     
    return salary
resume_dates = pd.to_datetime(fixed_df['Обновление резюме']).dt.date
mode_resume_date = resume_dates.mode()[0]
print('Mode date: ', mode_resume_date)
rates = ExchangeRates(mode_resume_date)

fixed_df['ЗП'].apply(lambda x: getParameterSalary(x, rates))

Mode date:  2019-07-05


0        40000
1        40000
2       300000
3       180000
4        40000
         ...  
996      20000
997      55000
998     180000
999      50000
1000     65000
Name: ЗП, Length: 1001, dtype: int64

In [6]:
rates['KZT'].rate

Decimal('0.165485')

1. Вытащить из DataFrame признаки пола и возраста 
2. Вытащить опыт работы в месяцах
3. Вытащить уровень образования (хардовая задача). Использовать регулярные выражения
4. Вытащить месяц и год даты обновления резюме
5. Перекодировать столбец Авто по типу [1, 0]

In [28]:
# # 1.1 Вытащить из DataFrame признаки пола
# age_sex=fixed_df['Пол, возраст'].str.split(', ')
# #age_sex
# sex_list=[]
# for sex in age_sex:
#     sex_list.append(sex[0])
# #print(sex_list)
# fixed_df['Sex']=sex_list
# fixed_df.head(1)

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,Sex,Age
0,0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,0,Мужчина,29


In [36]:
agesex=fixed_df.loc[0]['Пол, возраст']
agesex

'Мужчина ,  29 лет , родился 16 мая 1989'

Написать функцию, которая принимает в себя пол и возраст, а возвращает пол.

In [55]:
def get_sex(agesex):
    agesex_spl=agesex.split(' , ')
    return agesex_spl[0]
sex(agesex)

'Мужчина'

In [53]:
def get_age(agesex):
    agesex_spl=agesex.split(' , ')
    num = agesex_spl[1]
    num = re.sub(' ','',num)
    num = re.sub('[а-яА-ЯёЁ]','',num)
    num = re.sub('[a-zA-Z]','',num)
    num = num.replace('.','')
    return int(num)
get_age(agesex)


29

In [56]:
fixed_df['Sex']=fixed_df['Пол, возраст'].apply(get_sex)
fixed_df['Age']=fixed_df['Пол, возраст'].apply(get_age)
fixed_df.head()

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,Sex,Age
0,0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,0,Мужчина,29
1,1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,0,Мужчина,38
2,2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,0,Мужчина,35
3,3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,1,Мужчина,33
4,4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,0,Мужчина,22


In [57]:
# '''age_large=[]
# for age in age_sex:
#     age_large.append(int(age[1][1:3]))
# fixed_df['Age']=age_large
# fixed_df.head()#for figures in age_large:
# #   print (figures[1:3])'''

"age_large=[]\nfor age in age_sex:\n    age_large.append(int(age[1][1:3]))\nfixed_df['Age']=age_large\nfixed_df.head()#for figures in age_large:\n#   print (figures[1:3])"

In [9]:
# Вытащить опыт работы в месяцах
seniority_splitted=fixed_df['Опыт (двойное нажатие для полной версии)'].str.split(' ')
seniority_splitted.head(200)

0      [Опыт, работы, 3, года, 9, месяцев, , Специали...
1      [Опыт, работы, 11, лет, 11, месяцев, , Системн...
2      [Опыт, работы, 12, лет, 11, месяцев, , DevOps,...
3      [Опыт, работы, 15, лет, 10, месяцев, , Руковод...
4      [Опыт, работы, 1, год, 1, месяц, , Junior, Dev...
                             ...                        
195    [Опыт, работы, 4, года, 11, месяцев, , Июль, 2...
196    [Опыт, работы, 7, лет, 3, месяца, , Администра...
197    [Опыт, работы, 3, года, 11, месяцев, , Декабрь...
198    [Опыт, работы, 9, лет, 8, месяцев, , Август, 2...
199    [Опыт, работы, 15, лет, 7, месяцев, , Февраль,...
Name: Опыт (двойное нажатие для полной версии), Length: 200, dtype: object

In [58]:
# num_moths=[]
# i=0
# for elem in seniority_splitted:
#     i+=1
# #    if elem[2] is nan:
# #        continue
# #    elif elem[4] is nan:
# #        continue
#     if elem[2] =='':
#         print(float(elem[4]),i)
#     elif elem[4] =='':
#         continue
#     else:  
#         print((float(elem[2]))*12 + float(elem[4]), i)

        

1) Найдём индекс элемента "год", 'лет', 'года'

2) Найдём индекс элемента "месяцев", "месяц"

Перед этими словами будет стоять число, равное числу лет или месяцев.

In [78]:
experience=fixed_df.loc[0]['Опыт (двойное нажатие для полной версии)']
experience
def get_years(experience):
    search_words=['год', 'года', 'лет']
    experience_splitted = experience.split(' ')[:7]
   # print (experience_splitted)
    for index, item in enumerate (experience_splitted):
        if item in  search_words:
            return int(experience_splitted[index-1])
    return 0
get_years(experience)    

3

In [75]:
fixed_df.loc[0]['Опыт (двойное нажатие для полной версии)']

'Опыт работы 3 года 9 месяцев  Специалист по поддержке чата(support team) домашний агент 40 000 руб. Информационные технологии, интернет, телеком Системный администратор Сотовые, Беспроводные технологии Телекоммуникации Занятость: полная занятость График работы: полный день Опыт работы 3 года 9 месяцев Май 2012 — Январь  2016 3 года 9 месяцев ООО "Гольфстрим" Республика Башкортостан Строительство, недвижимость, эксплуатация, проектирование ... Строительство жилищное Управление и эксплуатация недвижимости Генеральный директор Руководство, отладка деловых связей, заключение договоров, развитие организации Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Заключение договоров Деловое общение Деловая коммуникация Пользователь ПК Ведение переговоров Управление персоналом Деловая переписка Составление договоров Телефонные переговоры Грамотная речь MS Outlook Водительское удостоверение категории B Работа в команде Возникли неполадки. Попробуйте еще

In [82]:
def get_months(experience):
    search_words=['месяц', 'месяца', 'месяцев']
    print(experience)
    experience_splitted = experience.split(' ')[:7]
    for index, item in enumerate (experience_splitted):
        if item in  search_words:
            return int(experience_splitted[index-1])
    return 0

get_months(experience)    

Опыт работы 3 года 9 месяцев  Специалист по поддержке чата(support team) домашний агент 40 000 руб. Информационные технологии, интернет, телеком Системный администратор Сотовые, Беспроводные технологии Телекоммуникации Занятость: полная занятость График работы: полный день Опыт работы 3 года 9 месяцев Май 2012 — Январь  2016 3 года 9 месяцев ООО "Гольфстрим" Республика Башкортостан Строительство, недвижимость, эксплуатация, проектирование ... Строительство жилищное Управление и эксплуатация недвижимости Генеральный директор Руководство, отладка деловых связей, заключение договоров, развитие организации Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Заключение договоров Деловое общение Деловая коммуникация Пользователь ПК Ведение переговоров Управление персоналом Деловая переписка Составление договоров Телефонные переговоры Грамотная речь MS Outlook Водительское удостоверение категории B Работа в команде Возникли неполадки. Попробуйте еще 

9

In [94]:
def get_experience(experience):
    if experience=='nan':
        return np.nan
    elif experience=='nan ':
        return np.nan
    months=get_months(experience)
    years=get_years(experience)

    experience_months=int(years*12+months)
    return experience_months

In [90]:
get_experience(experience)

Опыт работы 3 года 9 месяцев  Специалист по поддержке чата(support team) домашний агент 40 000 руб. Информационные технологии, интернет, телеком Системный администратор Сотовые, Беспроводные технологии Телекоммуникации Занятость: полная занятость График работы: полный день Опыт работы 3 года 9 месяцев Май 2012 — Январь  2016 3 года 9 месяцев ООО "Гольфстрим" Республика Башкортостан Строительство, недвижимость, эксплуатация, проектирование ... Строительство жилищное Управление и эксплуатация недвижимости Генеральный директор Руководство, отладка деловых связей, заключение договоров, развитие организации Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Заключение договоров Деловое общение Деловая коммуникация Пользователь ПК Ведение переговоров Управление персоналом Деловая переписка Составление договоров Телефонные переговоры Грамотная речь MS Outlook Водительское удостоверение категории B Работа в команде Возникли неполадки. Попробуйте еще 

45

In [95]:
fixed_df['Experience_months']=fixed_df['Опыт (двойное нажатие для полной версии)'].apply(get_experience)
fixed_df

Опыт работы 3 года 9 месяцев  Специалист по поддержке чата(support team) домашний агент 40 000 руб. Информационные технологии, интернет, телеком Системный администратор Сотовые, Беспроводные технологии Телекоммуникации Занятость: полная занятость График работы: полный день Опыт работы 3 года 9 месяцев Май 2012 — Январь  2016 3 года 9 месяцев ООО "Гольфстрим" Республика Башкортостан Строительство, недвижимость, эксплуатация, проектирование ... Строительство жилищное Управление и эксплуатация недвижимости Генеральный директор Руководство, отладка деловых связей, заключение договоров, развитие организации Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Заключение договоров Деловое общение Деловая коммуникация Пользователь ПК Ведение переговоров Управление персоналом Деловая переписка Составление договоров Телефонные переговоры Грамотная речь MS Outlook Водительское удостоверение категории B Работа в команде Возникли неполадки. Попробуйте еще 

AttributeError: 'float' object has no attribute 'split'

In [62]:
fixed_df.loc[0]['Опыт (двойное нажатие для полной версии)']

'Опыт работы 3 года 9 месяцев  Специалист по поддержке чата(support team) домашний агент 40 000 руб. Информационные технологии, интернет, телеком Системный администратор Сотовые, Беспроводные технологии Телекоммуникации Занятость: полная занятость График работы: полный день Опыт работы 3 года 9 месяцев Май 2012 — Январь  2016 3 года 9 месяцев ООО "Гольфстрим" Республика Башкортостан Строительство, недвижимость, эксплуатация, проектирование ... Строительство жилищное Управление и эксплуатация недвижимости Генеральный директор Руководство, отладка деловых связей, заключение договоров, развитие организации Ключевые навыки Теперь резюме открыто всему интернету — изменить можно в настройках видимости. Заключение договоров Деловое общение Деловая коммуникация Пользователь ПК Ведение переговоров Управление персоналом Деловая переписка Составление договоров Телефонные переговоры Грамотная речь MS Outlook Водительское удостоверение категории B Работа в команде Возникли неполадки. Попробуйте еще

In [11]:
seniority_splitted[0]

['Опыт',
 'работы',
 '3',
 'года',
 '9',
 'месяцев',
 '',
 'Специалист',
 'по',
 'поддержке',
 'чата(support',
 'team)',
 'домашний',
 'агент',
 '40',
 '000',
 'руб.',
 'Информационные',
 'технологии,',
 'интернет,',
 'телеком',
 'Системный',
 'администратор',
 'Сотовые,',
 'Беспроводные',
 'технологии',
 'Телекоммуникации',
 'Занятость:',
 'полная',
 'занятость',
 'График',
 'работы:',
 'полный',
 'день',
 'Опыт',
 'работы',
 '3',
 'года',
 '9',
 'месяцев',
 'Май',
 '2012',
 '—',
 'Январь',
 '',
 '2016',
 '3',
 'года',
 '9',
 'месяцев',
 'ООО',
 '"Гольфстрим"',
 'Республика',
 'Башкортостан',
 'Строительство,',
 'недвижимость,',
 'эксплуатация,',
 'проектирование',
 '...',
 'Строительство',
 'жилищное',
 'Управление',
 'и',
 'эксплуатация',
 'недвижимости',
 'Генеральный',
 'директор',
 'Руководство,',
 'отладка',
 'деловых',
 'связей,',
 'заключение',
 'договоров,',
 'развитие',
 'организации',
 'Ключевые',
 'навыки',
 'Теперь',
 'резюме',
 'открыто',
 'всему',
 'интернету',
 '—',
 '

In [12]:
#Вопросы Андрею. Как пройттись по списку внутри age[1]? 
for age in age_sex:
    print(age[1])

 29 лет 
 38 лет 
 35 лет 
 33 года 
 22 года 
 32 года 
 35 лет 
 23 года 
 28 лет 
 32 года 
 61 год 
 49 лет 
 54 года 
 25 лет 
 27 лет 
 27 лет 
 53 года 
 23 года 
 41 год 
 22 года 
 33 года 
 44 года 
 43 года 
 31 год 
 30 лет 
 33 года 
 34 года 
 30 лет 
 15 лет 
 27 лет 
 31 год 
 32 года 
 32 года 
 35 лет 
 23 года 
 32 года 
 43 года 
 35 лет 
 33 года 
 40 лет 
 37 лет 
 39 лет 
 31 год 
 32 года 
 32 года 
 29 лет 
 31 год 
 26 лет 
 29 лет 
 24 года 
 25 лет 
 31 год 
 34 года 
 42 года 
 27 лет 
 29 лет 
 27 лет 
 45 лет 
 25 лет 
 30 лет 
 27 лет 
 36 лет 
 36 лет 
 29 лет 
 30 лет 
 29 лет 
 45 лет 
 32 года 
 22 года 
 35 лет 
 56 лет 
 24 года 
 28 лет 
 20 лет 
 29 лет 
 36 лет 
 41 год 
 30 лет 
 32 года 
 31 год 
 38 лет 
 38 лет 
 33 года 
 23 года 
 20 лет 
 21 год 
 37 лет 
 32 года 
 34 года 
 27 лет 
 33 года 
 34 года 
 32 года 
 32 года 
 22 года 
 19 лет 
 54 года 
 38 лет 
 34 года 
 34 года 
 27 лет 
 31 год 
 30 лет 
 24 года 
 44 года 
 25 лет 
 43

In [20]:
# Перекодировать столбец Авто по типу [1, 0]
fixed_df['Авто'].value_counts()

Не указано                        730
Имеется собственный автомобиль    271
Name: Авто, dtype: int64

In [22]:
fixed_df['Авто']=pd.get_dummies(fixed_df['Авто'])
fixed_df

,Unnamed: 0,"Пол, возраст",ЗП,Ищет работу на должность:,Город,Занятость,График,Опыт (двойное нажатие для полной версии),Последенее/нынешнее место работы,Последеняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,Sex,Age
0,0,"Мужчина , 29 лет , родился 16 мая 1989",40000 руб.,Специалист по поддержке чата(support team) дом...,"Новороссийск , готов к переезду (Анапа, Геленд...",полная занятость,полный день,Опыт работы 3 года 9 месяцев Специалист по по...,"ООО ""Гольфстрим""",Генеральный директор,Высшее образование 2011 Международный юридиче...,26.04.2019 08:04,0,Мужчина,29
1,1,"Мужчина , 38 лет , родился 25 мая 1980",40000 руб.,Системный администратор,"Новосибирск , м. Березовая роща , не готов к ...",полная занятость,полный день,Опыт работы 11 лет 11 месяцев Системный админ...,ООО «Завод модульных технологий»,Системный администратор,Высшее образование 2002 Новосибирский государс...,26.04.2019 04:30,0,Мужчина,38
2,2,"Мужчина , 35 лет , родился 14 июня 1983",300000 руб.,DevOps TeamLead / DevOps архитектор,"Москва , готов к переезду , готов к редким ком...",полная занятость,полный день,Опыт работы 12 лет 11 месяцев DevOps TeamLead...,Банк ВТБ (ПАО),Начальник отдела методологии разработки (DevOp...,DevOps TeamLead / DevOps архитектор 300 000 ру...,09.04.2019 14:40,0,Мужчина,35
3,3,"Мужчина , 33 года , родился 2 августа 1985",180000 руб.,Руководитель IT отдела,"Москва , м. Щукинская , не готов к переезду ,...","частичная занятость, полная занятость","удаленная работа, полный день",Опыт работы 15 лет 10 месяцев Руководитель IT...,"""Ай-Теко"", ведущий российский системный интегр...",Старший системный администратор,Руководитель IT отдела 180 000 руб. Информацио...,09.04.2019 14:39,1,Мужчина,33
4,4,"Мужчина , 22 года , родился 1 сентября 1996",40000 руб.,Junior Developer,"Москва , м. Юго-Западная , не готов к переезд...","стажировка, частичная занятость, проектная работа","гибкий график, удаленная работа",Опыт работы 1 год 1 месяц Junior Developer 40...,R-Style SoftLab,Менеджер IT-проектов,Junior Developer 40 000 руб. Информационные те...,29.03.2019 12:40,0,Мужчина,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,996,"Мужчина , 33 года , родился 8 июня 1985",20000 руб.,"Мастер по ремонту ПК и орг техники (принтеры, ...","Великий Новгород , не готов к переезду , не го...",полная занятость,"удаленная работа, полный день, сменный график",Опыт работы 5 лет 8 месяцев Октябрь 2013 — по...,ООО Э-ТРИ,Мастер по ремонту ПК и орг техники,Образование Среднее образование,07.05.2019 01:07,0,Мужчина,33
997,997,"Мужчина , 39 лет , родился 18 мая 1979",55000 руб.,Менеджер по продажам,"Москва , не готов к переезду , готов к редким ...",полная занятость,полный день,Опыт работы 16 лет 4 месяца Октябрь 2017 — Се...,"ООО Компания "" Ипотека Полного Цикла""",Менеджер по работе с клиентами,Высшее образование 2002 Российский Новый Унив...,07.05.2019 00:12,0,Мужчина,39
998,998,"Мужчина , 43 года , родился 1 ноября 1975",180000 руб.,Руководитель ИТ-проектов,"Москва , готов к переезду , готов к командировкам","частичная занятость, проектная работа, полная ...","гибкий график, полный день",Опыт работы 17 лет 11 месяцев Руководитель ИТ...,Пенсионный фонд Российской Федерации,Начальник отдела проектирования ИТ-инфраструктуры,Руководитель ИТ-проектов 180 000 руб. Информац...,23.03.2019 18:31,1,Мужчина,43
999,999,"Женщина , 33 года , родилась 16 декабря 1985",50000 руб.,Наборщик текстов,"Москва , м. Жулебино , не готова к переезду ,...",полная занятость,удаленная работа,Опыт работы 12 лет 6 месяцев Ноябрь 2011 — по...,ООО Малти-Системс Текнолоджи,Руководитель отдела по работе с клиентами,Среднее специальное образование 2004 НОУ &quot...,12.04.2019 16:39,0,Женщина,33


ДЗ удалить наны
Последнее - вытащить уровень образования. 